In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import numpy as np

from selenium.webdriver.chrome.service import Service


def get_driver(use_proxy=False):

    chrome_options = Options()
    chrome_options = webdriver.ChromeOptions()

    if use_proxy:
        driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
        driver.get("https://sslproxies.org/")
        driver.execute_script("return arguments[0].scrollIntoView(true);", WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//table[@class='table table-striped table-bordered dataTable']//th[contains(., 'IP Address')]"))))
        ips = [my_elem.get_attribute("innerHTML") for my_elem in WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located((By.XPATH, "//table[@class='table table-striped table-bordered dataTable']//tbody//tr[@role='row']/td[position() = 1]")))]
        ports = [my_elem.get_attribute("innerHTML") for my_elem in WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located((By.XPATH, "//table[@class='table table-striped table-bordered dataTable']//tbody//tr[@role='row']/td[position() = 2]")))]
        proxies = []
        for i in range(0, len(ips)):
            proxies.append(ips[i]+':'+ports[i])
        chrome_options = webdriver.ChromeOptions()
        i = int(np.random.randint(low=0, high=5, size=(1,)))
        chrome_options.add_argument('--proxy-server={}'.format(proxies[i]))
        chrome_options.add_argument("start-maximized")
        chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
        chrome_options.add_experimental_option('useAutomationExtension', False)

    chrome_options.add_argument("start-maximized")
    service = Service()
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source":
        "const newProto = navigator.__proto__;"
        "delete newProto.webdriver;"
        "navigator.__proto__ = newProto;"
    })

    return driver

In [4]:
driver = get_driver()

urls = ["https://paperswithcode.com/"]

In [5]:
driver.get(urls[0])


In [6]:
html = driver.page_source
soup = BeautifulSoup(html)

In [9]:
soup.find_all("div", class_ = "col-lg-9 item-content")[0].find_all("a")[0].get("href")


'/paper/llama-omni-seamless-speech-interaction-with'

In [10]:
soup.find_all("div", class_ = "col-lg-9 item-content")[0].find_all("span", class_="author-name-text item-date-pub")[0].text

'10 Sep 2024'

In [11]:
import datetime

datetime.datetime.strptime(soup.find_all("div", class_ = "col-lg-9 item-content")[0].find_all("span", class_="author-name-text item-date-pub")[0].text, "%d %b %Y")


datetime.datetime(2024, 9, 10, 0, 0)

In [21]:
", ".join(f"{k}: {v}" for k,v in {"stars":soup.find_all("div", class_ = "entity-stars")[0].text.strip()}.items())


'stars: 834'

In [9]:
soup.find_all("div", class_ = "row infinite-item item paper-card")[0].find_all("a")[1].text


'LLaMA-Omni: Seamless Speech Interaction with Large Language Models'

In [9]:
soup.find_all("div", class_ = "col-lg-9 item-content")[0].find_all("p", class_="item-strip-abstract")[0].text




'We build our model based on the latest Llama-3. 1-8B-Instruct model.'

In [10]:
bookmarks = []
for element in soup.find_all("div", class_ = "col-lg-9 item-content"):
    print(element.find_all("a")[0].get("href"))
    print(element.find_all("a")[0].text)
    print(element.find_all("p", class_="item-strip-abstract")[0].text)
    bookmarks.append(
        {
            "url": element.find_all("a")[0].get("href"),
            "title": element.find_all("a")[0].text,
            "description": element.find_all("p", class_="item-strip-abstract")[0].text,
        }
    )

/paper/llama-omni-seamless-speech-interaction-with
LLaMA-Omni: Seamless Speech Interaction with Large Language Models
We build our model based on the latest Llama-3. 1-8B-Instruct model.
/paper/pulid-pure-and-lightning-id-customization-via
PuLID: Pure and Lightning ID Customization via Contrastive Alignment
We propose Pure and Lightning ID customization (PuLID), a novel tuning-free ID customization method for text-to-image generation.
/paper/paperqa-retrieval-augmented-generative-agent
PaperQA: Retrieval-Augmented Generative Agent for Scientific Research
We present PaperQA, a RAG agent for answering questions over the scientific literature.
/paper/windows-agent-arena-evaluating-multi-modal-os
Windows Agent Arena: Evaluating Multi-Modal OS Agents at Scale
To demonstrate Windows Agent Arena's capabilities, we also introduce a new multi-modal agent, Navi.
https://github.com/codelion/optillm
optillm
Optimizing inference proxy for LLMs
/paper/mini-omni-language-models-can-hear-talk-while
Mi

In [11]:
import polars as pl

pl.DataFrame(bookmarks)


url,title,description
str,str,str
"""/paper/llama-omni-seamless-spe…","""LLaMA-Omni: Seamless Speech In…","""We build our model based on th…"
"""/paper/pulid-pure-and-lightnin…","""PuLID: Pure and Lightning ID C…","""We propose Pure and Lightning …"
"""/paper/paperqa-retrieval-augme…","""PaperQA: Retrieval-Augmented G…","""We present PaperQA, a RAG agen…"
"""/paper/windows-agent-arena-eva…","""Windows Agent Arena: Evaluatin…","""To demonstrate Windows Agent A…"
"""https://github.com/codelion/op…","""optillm""","""Optimizing inference proxy for…"
"""/paper/mini-omni-language-mode…","""Mini-Omni: Language Models Can…","""We also introduce the VoiceAss…"
"""/paper/agent-workflow-memory""","""Agent Workflow Memory""","""Despite the potential of langu…"
"""/paper/hi3d-pursuing-high-reso…","""Hi3D: Pursuing High-Resolution…","""Despite having tremendous prog…"
"""/paper/do-anything-now-charact…","""""Do Anything Now"": Characteriz…","""We hope that our study can fac…"


## Create a component

In [12]:
from pydantic import BaseModel

class WebpageIntegration(BaseModel):
    base_urls: list[str]
    driver: webdriver.Chrome = None
    soup: BeautifulSoup = None

    model_config = {
        "arbitrary_types_allowed": True
    }

    def __init__(self, **data):
        super().__init__(**data)
        if self.driver is None:
            self.driver = get_driver()

    def load_page(self, url: str):
        self.driver.get(url)
        html = self.driver.page_source
        self.soup = BeautifulSoup(html, 'html.parser')

    def get_element_by_selector(self, selector: str, attr: str = None):
        element = self.soup.select_one(selector)
        if element:
            return element.get(attr) if attr else element.text.strip()
        return None

    def get_elements_by_selector(self, selector: str, attr: str = None):
        elements = self.soup.select(selector)
        if attr:
            return [element.get(attr) for element in elements if element.get(attr)]
        return [element.text.strip() for element in elements]

    def get_body(self):
        return self.get_element_by_selector('body')
    
    def _compose_bookmark(self, element):
        return {
            "url": element.find_all("a")[0].get("href"),
            "title": element.find_all("a")[0].text,
            "description": element.find_all("p", class_="item-strip-abstract")[0].text,
        }
    
    def get_bookmarks(self):
        bookmarks = []
        for base_url in self.base_urls:
            self.load_page(base_url)
            for element in self.soup.find_all("div", class_ = "col-lg-9 item-content"):
                bookmarks.append(self._compose_bookmark(element))
        return bookmarks
    
    def close(self):
        if self.driver:
            self.driver.quit()


In [14]:
wbi = WebpageIntegration(base_urls=urls, driver=driver)


In [15]:
l_bookmarks = wbi.get_bookmarks()

pl.DataFrame(l_bookmarks)


url,title,description
str,str,str
"""/paper/llama-omni-seamless-spe…","""LLaMA-Omni: Seamless Speech In…","""We build our model based on th…"
"""/paper/pulid-pure-and-lightnin…","""PuLID: Pure and Lightning ID C…","""We propose Pure and Lightning …"
"""/paper/paperqa-retrieval-augme…","""PaperQA: Retrieval-Augmented G…","""We present PaperQA, a RAG agen…"
"""/paper/windows-agent-arena-eva…","""Windows Agent Arena: Evaluatin…","""To demonstrate Windows Agent A…"
"""https://github.com/codelion/op…","""optillm""","""Optimizing inference proxy for…"
"""/paper/mini-omni-language-mode…","""Mini-Omni: Language Models Can…","""We also introduce the VoiceAss…"
"""/paper/agent-workflow-memory""","""Agent Workflow Memory""","""Despite the potential of langu…"
"""/paper/hi3d-pursuing-high-reso…","""Hi3D: Pursuing High-Resolution…","""Despite having tremendous prog…"
"""/paper/do-anything-now-charact…","""""Do Anything Now"": Characteriz…","""We hope that our study can fac…"


## Import current implementation

In [2]:
from easy_bookmarks.integrations.webpage import PwCWebpageIntegration

pwc = PwCWebpageIntegration(n_pages=5)

l_bookmarks = pwc.get_bookmarks()



There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache
Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)


In [2]:
pwc.base_urls

['https://paperswithcode.com/?page=1']

In [3]:
df = l_bookmarks.to_polars_dataframe()
df

url,title,description,reference_date,extra_data,uuid
str,str,str,datetime[μs],str,str
"""/paper/llama-omni-seamless-spe…","""LLaMA-Omni: Seamless Speech In…","""We build our model based on th…",2024-09-10 00:00:00,"""stars: 834""","""762e1210-da4e-ea52-e309-836833…"
"""/paper/pulid-pure-and-lightnin…","""PuLID: Pure and Lightning ID C…","""We propose Pure and Lightning …",2024-04-24 00:00:00,"""stars: 1,456""","""104af17e-0771-1ffe-408e-621fae…"
"""/paper/paperqa-retrieval-augme…","""PaperQA: Retrieval-Augmented G…","""We present PaperQA, a RAG agen…",2023-12-08 00:00:00,"""stars: 5,319""","""be7881d6-419b-c442-c4f0-e9672f…"
"""/paper/windows-agent-arena-eva…","""Windows Agent Arena: Evaluatin…","""To demonstrate Windows Agent A…",2024-09-12 00:00:00,"""stars: 115""","""02546c1c-ce9a-bbb8-5f13-0be3b2…"
"""https://github.com/codelion/op…","""optillm""","""Optimizing inference proxy for…",2024-09-05 00:00:00,"""stars: 336""","""f092e519-66d1-56e6-d59a-8117df…"
…,…,…,…,…,…
"""/paper/ntire-2024-challenge-on…","""NTIRE 2024 Challenge on Low Li…","""This paper reviews the NTIRE 2…",2024-04-22 00:00:00,"""stars: 632""","""e7ace71d-8475-b6a2-3a69-b82808…"
"""/paper/bench2drive-towards-mul…","""Bench2Drive: Towards Multi-Abi…","""In an era marked by the rapid …",2024-06-06 00:00:00,"""stars: 1,130""","""ecc41f72-5aa7-6b56-4f15-261f93…"
"""/paper/selective-reflection-tu…","""Selective Reflection-Tuning: S…","""This paper introduces Selectiv…",2024-02-15 00:00:00,"""stars: 304""","""a0322d45-6c92-6e1d-f3db-1f4d4f…"


In [4]:
df.write_csv("df_pwc.csv")

In [1]:
from easy_bookmarks.integrations.webpage import PwCWebpageIntegration

pwc = PwCWebpageIntegration(base_urls=["https://paperswithcode.com/latest/"], n_pages=5, driver=pwc.driver)

l_bookmarks = pwc.get_bookmarks()

NameError: name 'pwc' is not defined

In [3]:
pwc.base_urls

['https://paperswithcode.com/latest/?page=1']

In [4]:
l_bookmarks.to_polars_dataframe()

url,title,description,reference_date,uuid
str,str,str,datetime[μs],str
"""/paper/aceparse-a-comprehensiv…","""AceParse: A Comprehensive Data…","""With the development of data-c…",2024-09-16 00:00:00,"""3bf87fc3-dcd7-dffa-c82f-89575d…"
"""/paper/biomimetic-frontend-for…","""Biomimetic Frontend for Differ…","""While models in audio and spee…",2024-09-13 00:00:00,"""df1484bb-2a96-dcc2-f76b-cd057f…"
"""/paper/casdyf-net-image-dehazi…","""CasDyF-Net: Image Dehazing via…","""Inspired by dynamic filtering,…",2024-09-13 00:00:00,"""4ca610d4-b50d-31a0-4903-58a608…"
"""/paper/shadow-program-inversio…","""Shadow Program Inversion with …","""This paper presents SPI-DP, a …",2024-09-13 00:00:00,"""71752aee-10a0-2e39-98ce-ac8139…"
"""/paper/when-context-leads-but-…","""When Context Leads but Paramet…","""Large language models (LLMs) h…",2024-09-13 00:00:00,"""0350922f-a714-a0ea-9c75-99f78d…"
"""/paper/integrating-neural-oper…","""Integrating Neural Operators w…","""We integrate neural operators …",2024-09-13 00:00:00,"""1c62bf7a-7427-2923-69aa-67e801…"
"""/paper/exploiting-supervised-p…","""Exploiting Supervised Poison V…","""Availability poisons exploit s…",2024-09-13 00:00:00,"""60ee20a5-9fd8-1d84-ebae-8f0405…"
"""/paper/in-depth-analysis-of-lo…","""In-depth Analysis of Low-rank …","""We provide a linear rate of co…",2024-09-13 00:00:00,"""1c664a84-88e0-99c1-58bf-f13247…"
"""/paper/mapx-an-explainable-mod…","""MAPX: An explainable model-agn…","""The automated detection of fal…",2024-09-13 00:00:00,"""07daa2c8-d53e-df6b-52ff-3a886e…"
